# This python example will show
    -> How to achieve an token, on order to call the forma API
    -> Example of how to call various forma public API
    -> Get Project details, create and update Proposal, Create element, Update/Replace element, Get specific element
    -> Get element geometry, Create and get Terrain(Yet)

In [ ]:
# Importing all the required modules
# Install in not installed
# Example: pip install requests
import requests
import webbrowser as wb
import urllib
import pkce
from http.server import HTTPServer, BaseHTTPRequestHandler

In [ ]:

APS_APP_CLIENT_ID = 'ID'
REDIRECT_URI = 'APS_APP_REDIRECT_URI'
AUTHORIZER_URL = 'https://developer.api.autodesk.com/authentication/v2/authorize'
EDITOR_SCOPES = ["data:read", "data:write"] # Scope that allows the user to read and write data
LOCAL_SERVER = ('localhost', 8080) # Local server to handle the redirect, could be your own server
code_verifier = pkce.generate_code_verifier(length=128) # Generate a code verifier, required to generate token
code_challenge = pkce.get_code_challenge(code_verifier) # Generate a code challenge, required to generate token

In [ ]:
# Build the URL to open in the browser, as this will ask user to be authenticated and give consent to the scopes
AUTHORIZER_BROWSER_URL = f'{AUTHORIZER_URL}?response_type=code&client_id={APS_APP_CLIENT_ID}&redirect_uri={REDIRECT_URI}&scope={" ".join(EDITOR_SCOPES)}&code_challenge={code_challenge}&code_challenge_method=S256&nonce=1234'

In [ ]:
# Create a local server to handle the redirect
def create_server(host: str, post: str):
    class Server(BaseHTTPRequestHandler):
        def do_GET(self):
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            self.wfile.write(b'<html><body><h1>Authorization complete</h1></body></html>')
            if self.path.startswith(urllib.parse.urlparse(REDIRECT_URI).path):
                # Save the code in the local server object
                self.server.auth_code = urllib.parse.parse_qs(urllib.parse.urlparse(self.path).query)['code'][0]
                # Close the server
                self.server.shutdown()

    return HTTPServer((host, post), Server)

In [ ]:
# Create a server and open the browser in a new tab
# User user will be redirected to the APS_APP_REDIRECT_URI with the code as a query parameter
# Server will be closed after the user is redirected and the code is saved
server = create_server(*LOCAL_SERVER)
wb.open_new_tab(AUTHORIZER_BROWSER_URL)
server.serve_forever()

In [ ]:
# Get the token using the code
# The token can be used to make requests to the Autodesk API
response = requests.post('https://developer.api.autodesk.com/authentication/v2/token', 
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'accept': 'application/json'
    }, 
    data = {
        'grant_type': 'authorization_code',
        'client_id': APS_APP_CLIENT_ID,
        'code_verifier': code_verifier,
        'code': server.auth_code,
        'redirect_uri': REDIRECT_URI,
    }
).json()
print(response['access_token']) # Print the token

In [ ]:
## Get a project
# Get the project id from your Forma APP
PROJECT_ID = 'YOUR_PROJECT_ID'
project = requests.get(f'https://developer.api.autodesk.com/forma/project/v1alpha/projects/{PROJECT_ID}', headers = {
    'x-ads-region': 'EMEA',
    'Authorization': f'Bearer {response["access_token"]}',
}).json()
print(project) # Print the project

In [ ]:
## Create proposal
# Create a proposal with the project id and the token
# For creating the proposal, one has top pass the terrain and base urns
proposal = requests.post(f'https://developer.api.autodesk.com/forma/proposal/v1alpha/proposals?authcontext={PROJECT_ID}', 
    headers = {
        'x-ads-region': 'EMEA',
        'Authorization': f'Bearer {response["access_token"]}',
    }, 
    json = {
        'terrain': {
            'key': 'YOUR_TERRAIN_KEY',
            'urn': 'YOUR_TERRAIN_URN'
        },
        'base': {
            'urn': 'YOUR_BASE_URN',
            'key': 'YOUR_BASE_KEY'
        },
        'children': [],
        'name': 'YOUR PROPOSAL NAME',
    },
).json()
print(proposal)

In [ ]:
## Get a specif proposal by URN
# In forma, the proposal is an special kind of element, with various metadata
proposal_urn = proposal['urn']
proposal_element = requests.get(f'https://developer.api.autodesk.com/forma/element-service/v1alpha/elements/{proposal_urn}?authcontext={PROJECT_ID}', headers = {
    'x-ads-region': 'EMEA',
    'Authorization': f'Bearer {response["access_token"]}',
}).json()
print(proposal_element)

In [ ]:
## Get an element by URN
# This API will return the metadata of the element, actual geometry could be retrieved using the Representation API
element_urn_terrain = 'YOUR_ELEMENT_URN'
terrain_element = requests.get(f'https://developer.api.autodesk.com/forma/element-service/v1alpha/elements/{element_urn_terrain}?authcontext={PROJECT_ID}', headers = {
    'x-ads-region': 'EMEA',
    'Authorization': f'Bearer {response["access_token"]}',
}).json()
print(terrain_element)

In [ ]:
## Get a element volumeMash representation
# This is an example of getting a terrain element representation
# This API will return the actual geometry of the element
YOUR_TERRAIN_ELEMENT_URN = 'YOUR_TERRAIN_ELEMENT_URN'
terrain_element_representation = requests.get(f'https://developer.api.autodesk.com/forma/element-service/v1alpha/elements/{YOUR_TERRAIN_ELEMENT_URN}/representations/volumeMesh?authcontext={PROJECT_ID}', headers = {
    'x-ads-region': 'EMEA',
    'Authorization': f'Bearer {response["access_token"]}',
})
print(terrain_element_representation.text)

In [ ]:
## Get a element footprint representation
# This is an example of getting a footprint element representation
footprint_element_urn = 'YOUR_FOOTPRINT_ELEMENT_URN'
footprint_element_representation = requests.get(f'https://developer.api.autodesk.com/forma/element-service/v1alpha/elements/{footprint_element_urn}/representations/footprint?authcontext={PROJECT_ID}', headers = {
    'x-ads-region': 'EMEA',
    'Authorization': f'Bearer {response["access_token"]}',
})
print(footprint_element_representation.text)

### integrate API
### This API is used to store non Forma native geometry in the Forma

In [ ]:
## If the geometry is too big, you can use the S3 API to upload the file
## This is an example of getting an s3 link to upload a file
s3_link = requests.get(f'https://developer.api.autodesk.com/forma/integrate/v1alpha/upload-link?authcontext={PROJECT_ID}', headers= {
    'x-ads-region': 'EMEA',
    'Authorization': f'Bearer {response["access_token"]}',
}).json()
print(s3_link)

In [ ]:
## Get the file buffer, example: car.glb
## Upload the file to the S3 link
file = open('./car.glb', 'rb')
byte_array = file.read()
upload_file_result = requests.put(f'{s3_link["url"]}', data=byte_array)
print("File successfully uploaded" if upload_file_result.status_code == 200 else f'File upload failed, Reason = {upload_file_result.reason}')

In [ ]:
## Create an element
## This is an example of creating an element with the uploaded file
## In the geometry is inline geometry then, the geometry can be inserted directly without the need of uploading the file
upload_element = requests.post(f'https://developer.api.autodesk.com/forma/integrate/v1alpha/elements?authcontext={PROJECT_ID}', headers={
    'x-ads-region': 'EMEA',
    'Authorization': f'Bearer {response["access_token"]}',
    'Content-Type': 'application/json'
}, json= {
    'rootElement': 'root',
    'elements': {
        'root': {
            'id': 'root',
            'properties': {
                'geometry': {
                    'type': "File",
                    'format': "glb",
                    's3Id': s3_link["id"]
                }
            }
        }
    }
}).json()
print(upload_element)

In [ ]:
# Replace an existing element
# This is an example of replacing an existing element with the uploaded file
replace_element = requests.put(f'https://developer.api.autodesk.com/forma/integrate/v1alpha/elements/{footprint_element_urn}?authcontext={PROJECT_ID}', headers={
    'x-ads-region': 'EMEA',
    'Authorization': f'Bearer {response["access_token"]}',
    'Content-Type': 'application/json'
}, json={
    'rootElement': 'root',
    'elements': {
        'root': {
            'id': 'root',
            'properties': {
                'geometry': {
                    'type': 'File',
                    'format': 'glb',
                    's3Id': s3_link['id']
                }
            }
        }
    }
}).json()